<a href="https://colab.research.google.com/github/msphuonghuynh/pydata-notebook/blob/master/Build_A_Stock_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.youtube.com/watch?v=EYnC4ACIt2g

In [67]:
pip install quandl

In [68]:
import quandl
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [69]:
df = quandl.get("WIKI/FB")
print(df.head())

             Open   High    Low  ...  Adj. Low  Adj. Close  Adj. Volume
Date                             ...                                   
2012-05-18  42.05  45.00  38.00  ...     38.00     38.2318  573576400.0
2012-05-21  36.53  36.66  33.00  ...     33.00     34.0300  168192700.0
2012-05-22  32.61  33.59  30.94  ...     30.94     31.0000  101786600.0
2012-05-23  31.37  32.50  31.36  ...     31.36     32.0000   73600000.0
2012-05-24  32.95  33.21  31.77  ...     31.77     33.0300   50237200.0

[5 rows x 12 columns]


In [70]:
# Check the current directory and print it out
import os
print( os.getcwd() )

/content


In [71]:
# Mount the Google drive to talk with the root of colab
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [72]:
# Check the current directory and print it out
print( os.getcwd() )

/content


In [73]:
# Save dataframe to the root
df.to_csv('data.csv')

In [74]:
# Transfer data from root to Google Drive
!cp data.csv "/content/gdrive/My Drive"

In [75]:
# an example of writing a string
with open('/content/gdrive/My Drive/file.txt', 'w') as f:
  f.write("content")
# end this example

In [76]:
# Call the dataframe from the google drive
df2 = pd.read_csv("/content/gdrive/My Drive/data.csv")

In [77]:
df = df2
df=df.set_index('Date')
df

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2012-05-18,42.05,45.00,38.00,38.2318,573576400.0,0.0,1.0,42.05,45.00,38.00,38.2318,573576400.0
2012-05-21,36.53,36.66,33.00,34.0300,168192700.0,0.0,1.0,36.53,36.66,33.00,34.0300,168192700.0
2012-05-22,32.61,33.59,30.94,31.0000,101786600.0,0.0,1.0,32.61,33.59,30.94,31.0000,101786600.0
2012-05-23,31.37,32.50,31.36,32.0000,73600000.0,0.0,1.0,31.37,32.50,31.36,32.0000,73600000.0
2012-05-24,32.95,33.21,31.77,33.0300,50237200.0,0.0,1.0,32.95,33.21,31.77,33.0300,50237200.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2018-03-21,164.80,173.40,163.30,169.3900,105350867.0,0.0,1.0,164.80,173.40,163.30,169.3900,105350867.0
2018-03-22,166.13,170.27,163.72,164.8900,73389988.0,0.0,1.0,166.13,170.27,163.72,164.8900,73389988.0
2018-03-23,165.44,167.10,159.02,159.3900,52306891.0,0.0,1.0,165.44,167.10,159.02,159.3900,52306891.0


In [78]:
df = df[['Adj. Close']]
print(df.head())

            Adj. Close
Date                  
2012-05-18     38.2318
2012-05-21     34.0300
2012-05-22     31.0000
2012-05-23     32.0000
2012-05-24     33.0300


In [79]:
# A variable for predicting 'n' days out into the future
forecast_out = 30
# Create another column (the target or dependent variable) shifted 'n' units up
df['Prediction'] = df[['Adj. Close']].shift(-forecast_out)
# Print(df.head())
print(df.head())

            Adj. Close  Prediction
Date                              
2012-05-18     38.2318      30.771
2012-05-21     34.0300      31.200
2012-05-22     31.0000      31.470
2012-05-23     32.0000      31.730
2012-05-24     33.0300      32.170


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [80]:
print(df.tail())

            Adj. Close  Prediction
Date                              
2018-03-21      169.39         NaN
2018-03-22      164.89         NaN
2018-03-23      159.39         NaN
2018-03-26      160.06         NaN
2018-03-27      152.19         NaN


In [81]:
### Create the independent data set (X) ###
# Convert the datagrame to a numpy array
X = np.array(df.drop(['Prediction'],1))
# Remove the last 'n' row
X = X[:-forecast_out]
print(X)


[[ 38.2318]
 [ 34.03  ]
 [ 31.    ]
 ...
 [171.5499]
 [175.98  ]
 [176.41  ]]


In [82]:
### Create the dependent data set (y) ###
# Convert the datagrame to a numpy array (All of the values including the NaN's)
y = np.array(df['Prediction'])
# Get all of the y values except the last 'n'rows
y = y[:-forecast_out]
print(y)

[ 30.771  31.2    31.47  ... 159.39  160.06  152.19 ]


In [83]:
# Split the data into 80% training and 20% testing
x_train,x_test,y_train,y_test = train_test_split(X,y, test_size = 0.2)

In [84]:
# Create and train the Support Vector Machine (Regressor)
svr_rbf = SVR(kernel='rbf',C=1e3, gamma=0.1)
svr_rbf.fit(x_train, y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [85]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction
# The best possible score is 1.0
svm_confidence = svr_rbf.score(x_test,y_test)
print("svm confidence: ", svm_confidence)


svm confidence:  0.9824767216835069


In [86]:
# Create and train the Linear Regression Model
lr = LinearRegression()
# Train the Model
lr.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [87]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction
# The best possible score is 1.0
lr_confidence = lr.score(x_test,y_test)
print("lr confidence: ", lr_confidence)

lr confidence:  0.9792249025429927


In [88]:
# Set x_forecast equal to the last 30 rows of the original data set from Adj. Close column
x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]
print(x_forecast)

[[173.15]
 [179.52]
 [179.96]
 [177.36]
 [176.01]
 [177.91]
 [178.99]
 [183.29]
 [184.93]
 [181.46]
 [178.32]
 [175.94]
 [176.62]
 [180.4 ]
 [179.78]
 [183.71]
 [182.34]
 [185.23]
 [184.76]
 [181.88]
 [184.19]
 [183.86]
 [185.09]
 [172.56]
 [168.15]
 [169.39]
 [164.89]
 [159.39]
 [160.06]
 [152.19]]


In [89]:
# Print linear regression model predictions for the next 'n' days
lr_prediction = lr.predict(x_forecast)
print(lr_prediction)

# Print support vector regression model predictions for the next 'n' days
svm_prediction = svr_rbf.predict(x_forecast)
print(svm_prediction)

[177.08725448 183.52431676 183.968949   181.34157664 179.97736407
 181.89736695 182.98873701 187.33400668 188.99127232 185.48474075
 182.31168336 179.90662712 180.59378605 184.41358125 183.78705399
 187.75842837 186.37400524 189.29443067 188.81948259 185.90916244
 188.24348173 187.91000754 189.15295677 176.49104306 172.03461532
 173.28766983 168.74029459 163.18239153 163.85944517 155.90659115]
[176.9772862  181.21669313 179.89413781 174.5914663  171.63664029
 177.8475503  181.89126625 187.20647005 179.04419262 181.17386159
 180.11086532 171.6882011  171.89274521 178.83637012 180.45731349
 185.31177969 186.33961294 178.72933492 179.52391138 183.72822511
 182.38192958 184.43024422 178.79049423 178.3644724  171.31480923
 171.95800365 171.84232589 167.34492454 167.89151319 161.63256574]
